In [ ]:
import numpy as np
import cv2 as cv
from math import sqrt
import numpy as np
from PIL import Image

In [ ]:
def order_points(pts):
    
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

In [ ]:
def four_point_transform(image, pts):

    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    M = cv.getPerspectiveTransform(rect, dst)
    warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))
    
    return warped

In [ ]:
def Alignment(data, image):
    
    bboxes = data['boxes']
    segments = data['segments']
 
    bbox = bboxes[i]
    bbox = [int(bbox) for bbox in bbox]
    bbox = [0 if i < 0 else i for i in bbox]
    bbox = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
    segment_list = segments[i][0]
    segment_list = [int(segment_list) for segment_list in segment_list]
    segment_list = [0 if i < 0 else i for i in segment_list]
    x_segement = segment_list[0::2]
    y_segement = segment_list[1::2]
    segemnts_2d = np.array(list(zip(x_segement, y_segement)))
    pts = segemnts_2d
    
    warped = four_point_transform(image, pts)
        
    return warped

In [ ]:
warped = Alignment(data, image)

# Opencv to PIL conversion for GAN model 

# color_coverted = cv.cvtColor(warped, cv.COLOR_BGR2RGB)
# pil_image = Image.fromarray(color_coverted).convert('RGB')